In [1]:
ls

assets@             model.lp            repair.det.old.md   repair.md
eval.all.csv        models.ipynb        repair.det.pdf      repair.pdf
eval.py             models.stoc.ipynb   repair.det.tex      repair.stoc.md
gurobi.log          pyrp/               repair.det.tex.raw  repair.tex
instances/          repair.bib          repair.det.xdv      repair.tex.raw
log.log             repair.det.html     repair.html         repair.xdv
main.py             repair.det.md       repair.intros.md    test.sh


In [12]:
import importlib
import numpy as np 
from collections import namedtuple as nt

In [3]:
import pandas as pd
from pyrp.util import *
import importlib

In [20]:
kwargs = {"subproblem_alg": 'dp',
          "mp": True,
          "scale": 15}
  
num_i = 10
num_t = 15

mp = kwargs.get('mp', False)
subproblem_alg = kwargs.get('subproblem_alg', 'dp')

problem = create_instance_vec(num_i, num_t, 100)
h, b = problem['h'], problem['p']
T = problem['T']
I = problem['I']
D = problem['D']
numI = len(problem['I'])

# # query subproblem algorithm
# if subproblem_alg == 'mip':
#   # use mip
#   subproblem_method = pms.single_mip
# elif subproblem_alg == 'dp':
#   # use dp
#   subproblem_method = pmsd.single_dp
# else:
#   raise ValueError("unknown method for sub problem")

In [37]:
d_array = problem['Darr']
d_mu = np.mean(d_array, axis=0)
d_cov = np.cov(d_array, rowvar=False)

## Benchmark by Conic

With moment uncertainty for $\boldsymbol d: \mathbb{E}(\boldsymbol d) = \mu_0, ...$, in @delage_distributionally_2010. The DRO model is equivalent to the following problem:

$$\begin{aligned}
\min_{\boldsymbol{U}, \boldsymbol{Q}, \boldsymbol{\beta}, r, s} & \left(\gamma_{2} \boldsymbol{\Sigma}_{0}-\boldsymbol{\mu}_{0} \boldsymbol{\mu}_{0}^{\top}\right) \bullet \boldsymbol{Q}+r+\left(\boldsymbol{\Sigma}_{0} \bullet \boldsymbol{P}\right)-2 \boldsymbol{\mu}_{0}^{\top} \boldsymbol{p} + \gamma_{1} s \\
\mathbf { s.t. } & \\
& \boldsymbol{Q} \succeq 0, \boldsymbol{\beta} \in \mathbb{R}^{|T|}, 
\begin{bmatrix}
  \boldsymbol{P} & \boldsymbol{p} \\
  \boldsymbol{p}^\top & s
\end{bmatrix} \succeq 0, \; 
\boldsymbol \beta = 2 (\boldsymbol p + \boldsymbol{Q\mu}_0)\\
& \boldsymbol{U} ^\top e + \boldsymbol \delta^+ - \boldsymbol \delta^-  = \boldsymbol d & \forall \boldsymbol d \in \Xi_d\\
& \boldsymbol{d}^{\top} \boldsymbol{Q} \boldsymbol{d} -\boldsymbol{d^\top\beta} + r \ge e^\top( b \cdot\boldsymbol{\delta^+}  + h\cdot \boldsymbol \delta^-) & \forall \boldsymbol d \in \Xi_d \\
& \boldsymbol X_{(i,\cdot)}, \boldsymbol U_{(i,\cdot)}, \boldsymbol S_{(i,\cdot)} \in \Omega_i & \forall i\in I
\end{aligned}$$

semi-infinite constraints are equivalent to

$$\begin{bmatrix}
  \boldsymbol Q &  (he- \boldsymbol \beta)/2 \\
  (he- \boldsymbol \beta)^\top/2 & r - (h+b)e^\top \boldsymbol \delta^+ - h e^\top   \boldsymbol U^\top e
\end{bmatrix} \succeq 0$$

wrap up:

$$\begin{aligned}
\min_{\boldsymbol{x}, \boldsymbol{Q}, \boldsymbol{\beta}, r, s} & \left(\gamma_{2} \boldsymbol{\Sigma}_{0}-\boldsymbol{\mu}_{0} \boldsymbol{\mu}_{0}^{\top}\right) \bullet \boldsymbol{Q}+r+\left(\boldsymbol{\Sigma}_{0} \bullet \boldsymbol{P}\right)-2 \boldsymbol{\mu}_{0}^{\top} \boldsymbol{p} + \gamma_{1} s \\
\mathbf { s.t. } & \\
& \boldsymbol{Q} \succeq 0, \boldsymbol{\beta} \in \mathbb{R}^{|T|}, 
\begin{bmatrix}
  \boldsymbol{P} & \boldsymbol{p} \\
  \boldsymbol{p}^\top & s
\end{bmatrix} \succeq 0, \; 
\boldsymbol \beta = 2 (\boldsymbol p + \boldsymbol{Q\mu}_0)\\
& \begin{bmatrix}
  \boldsymbol Q &  (he- \boldsymbol \beta)/2 \\
  (he- \boldsymbol \beta)^\top/2 & r - (h+b)e^\top \boldsymbol \delta^+ - h e^\top   \boldsymbol U^\top e
\end{bmatrix} \succeq 0 \\
& \boldsymbol X_{(i,\cdot)}, \boldsymbol U_{(i,\cdot)}, \boldsymbol S_{(i,\cdot)} \in \Omega_i & \forall i\in I
\end{aligned}$$

In [39]:
import mosek.fusion as mf
expr = mf.Expr
dom = mf.Domain
mat = mf.Matrix

In [41]:
num_i, num_t

(10, 15)

In [ ]:
model = mf.Model("repair-wt-mmt-uncertainty")

S = model.variable([num_i, num_t], dom.greaterThan(0))
X = model.variable([num_i, num_t], dom.binary())
U = model.variable([num_i, num_t], dom.binary())
s = model.variable([num_i, num_t], dom.greaterThan(0))
P = model.variable([num_i, num_i], dom.inPSDCone())
p = model.variable([num_i])
beta = model.variable([num_i])